In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
from summer2 import Stratification, Multiply

import itertools

pd.options.plotting.backend = "plotly"


In [ ]:
class Compartment:
    """
    A tuberculosis model compartment.
    """
    SUSCEPTIBLE = "susceptible"
    EARLY_LATENT = "early_latent"
    LATE_LATENT = "late_latent"
    INFECTIOUS = "infectious"
    ON_TREATMENT = "on_treatment"
    RECOVERED = "recovered"



#REPLICABLE_COMPARTMENTS = [Compartment.LATENT, Compartment.INFECTIOUS]

BASE_COMPARTMENTS = [
    Compartment.SUSCEPTIBLE,
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
    Compartment.INFECTIOUS,
    Compartment.ON_TREATMENT,
    Compartment.RECOVERED,
]
INFECTIOUS_COMPS = [
    Compartment.INFECTIOUS,
    Compartment.ON_TREATMENT,
]

LATENT_COMPS = [
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
]

In [ ]:
model = CompartmentalModel(
    times=(1800, 2020),
    compartments=BASE_COMPARTMENTS,
    infectious_compartments=INFECTIOUS_COMPS,
    timestep=0.1,
)

In [ ]:
model.set_initial_population(
    distribution=
    {
        Compartment.INFECTIOUS: 9900.0, 
        Compartment.SUSCEPTIBLE: 100.0,
    },
)
age_groups= [0, 5, 15, 35, 50]


In [ ]:
model.add_infection_frequency_flow(
    name="infection",
    contact_rate=1,
    source=Compartment.SUSCEPTIBLE,
    dest=Compartment.EARLY_LATENT,
)

model.add_infection_frequency_flow(
    "infection_from_latent",
    0.3,
    Compartment.LATE_LATENT,
    Compartment.EARLY_LATENT,
)
model.add_infection_frequency_flow(
    "infection_from_recovered",
    0.5,
    Compartment.RECOVERED,
    Compartment.EARLY_LATENT,
)
model.add_transition_flow(
    "self_recovery",
    0.21,
    Compartment.INFECTIOUS,
    Compartment.RECOVERED,
)
model.add_death_flow(
    name="infection_death",
    death_rate=0.01,
    source=Compartment.INFECTIOUS
)

In [ ]:
early_activation_dict = {
    0: 2.4107,
    5: 0.9862,
    15: 0.0986,
}
late_activation_dict = {
    0: 6.939769e-9,
    5: 0.0023,
    15: 0.0012 ,
}
stabilisation_dict = {
    0: 4.383,
    5: 4.383,
    15: 1.972,
}

In [ ]:
#Mapping param values
def map_params(input_dict, targets):
    results = {}
    for t in targets:
        keys = [k for k in input_dict.keys() if k <= t]
        results[t] = input_dict[max(keys)] if keys else None
    results = {str(k): v for k,v in results.items()}
    return results


In [ ]:
stabilisation_rate = 1
early_activation_rate = 1
late_activation_rate = 1
model.add_transition_flow(
    "stabilisation",
    stabilisation_rate,
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
)
model.add_transition_flow(
    "early_activation",
    early_activation_rate,
    Compartment.EARLY_LATENT,
    Compartment.INFECTIOUS,
)
model.add_transition_flow(
    "late_activation",
    late_activation_rate,
    Compartment.LATE_LATENT,
    Compartment.INFECTIOUS,
    )
    # Add post-diseases flows
model.add_transition_flow(
    "self_recovery",
    0.21,
    Compartment.INFECTIOUS,
    Compartment.RECOVERED,
)

In [ ]:
def get_age_stratification() -> Stratification:
    # Create the stratification
    strat = Stratification(name="age", strata=age_groups, compartments=BASE_COMPARTMENTS)
    return strat
age_strat = get_age_stratification()
age_strat.set_flow_adjustments(
    "stabilisation", map_params(stabilisation_dict, age_groups)
)
age_strat.set_flow_adjustments(
    "early_activation", map_params(early_activation_dict, age_groups)
)
age_strat.set_flow_adjustments(
    "late_activation", map_params(late_activation_dict, age_groups)
)


In [ ]:
model.stratify_with(age_strat)

In [ ]:
model.request_output_for_flow("Early LTBI to infectious", "early_activation")
model.run()
outputs_df = model.get_outputs_df()
outputs_df.plot()